<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/langchain-course/06_RAG_Basic_Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb transformers gensim --quiet

## Working with **Embeddings**

In [ ]:
import gensim.downloader as api

# Load the GloVe model from Gensim-data repository
# Here we use 'glove-wiki-gigaword-50' as an example. Other dimensions/models are available as well.
glove_model = api.load('glove-wiki-gigaword-50')
word = 'python'
word_vector = glove_model[word]
print(word_vector.shape)

[==================================================] 100.0% 66.0/66.0MB downloaded
(50,)


In [ ]:
word_vector

array([ 0.5897  , -0.55043 , -1.0106  ,  0.41226 ,  0.57348 ,  0.23464 ,
       -0.35773 , -1.78    ,  0.10745 ,  0.74913 ,  0.45013 ,  1.0351  ,
        0.48348 ,  0.47954 ,  0.51908 , -0.15053 ,  0.32474 ,  1.0789  ,
       -0.90894 ,  0.42943 , -0.56388 ,  0.69961 ,  0.13501 ,  0.16557 ,
       -0.063592,  0.35435 ,  0.42819 ,  0.1536  , -0.47018 , -1.0935  ,
        1.361   , -0.80821 , -0.674   ,  1.2606  ,  0.29554 ,  1.0835  ,
        0.2444  , -1.1877  , -0.60203 , -0.068315,  0.66256 ,  0.45336 ,
       -1.0178  ,  0.68267 , -0.20788 , -0.73393 ,  1.2597  ,  0.15425 ,
       -0.93256 , -0.15025 ], dtype=float32)

In [ ]:
v1 = glove_model["king"]
v2 = glove_model['ruler']
v3 = glove_model['table']

In [ ]:
v1

array([ 0.50451 ,  0.68607 , -0.59517 , -0.022801,  0.60046 , -0.13498 ,
       -0.08813 ,  0.47377 , -0.61798 , -0.31012 , -0.076666,  1.493   ,
       -0.034189, -0.98173 ,  0.68229 ,  0.81722 , -0.51874 , -0.31503 ,
       -0.55809 ,  0.66421 ,  0.1961  , -0.13495 , -0.11476 , -0.30344 ,
        0.41177 , -2.223   , -1.0756  , -1.0783  , -0.34354 ,  0.33505 ,
        1.9927  , -0.04234 , -0.64319 ,  0.71125 ,  0.49159 ,  0.16754 ,
        0.34344 , -0.25663 , -0.8523  ,  0.1661  ,  0.40102 ,  1.1685  ,
       -1.0137  , -0.21585 , -0.15155 ,  0.78321 , -0.91241 , -1.6106  ,
       -0.64426 , -0.51042 ], dtype=float32)

In [ ]:
# prompt: python code to calculate similarity between two vectors

import numpy as np
similarity = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
print(similarity)


0.74342537


In [ ]:
similarity = np.dot(v1, v3) / (np.linalg.norm(v1) * np.linalg.norm(v3))
print(similarity)

0.2848273


In [ ]:
glove_model.most_similar("stocks")

[('stock', 0.8653818368911743),
 ('markets', 0.8522835969924927),
 ('prices', 0.8431004285812378),
 ('market', 0.8400351405143738),
 ('traders', 0.8257467150688171),
 ('trading', 0.8112872838973999),
 ('investors', 0.8083530068397522),
 ('indexes', 0.7902355194091797),
 ('dealers', 0.7884277701377869),
 ('shares', 0.7868536114692688)]

In [ ]:
glove_model.most_similar("trading")

[('stock', 0.9012669920921326),
 ('exchange', 0.898104190826416),
 ('futures', 0.8487032651901245),
 ('trades', 0.8236047029495239),
 ('traded', 0.8166490793228149),
 ('stocks', 0.8112873435020447),
 ('market', 0.8051413893699646),
 ('prices', 0.7966799139976501),
 ('closing', 0.7950035929679871),
 ('closed', 0.7914804220199585)]

In [ ]:
glove_model.most_similar("amazing")

[('incredible', 0.9189565181732178),
 ('fantastic', 0.8799790143966675),
 ('awesome', 0.8620665669441223),
 ('wonderful', 0.8537988662719727),
 ('terrific', 0.8482187390327454),
 ('marvelous', 0.8439217805862427),
 ('astonishing', 0.8103041052818298),
 ('remarkable', 0.8091045022010803),
 ('exciting', 0.79411780834198),
 ('unbelievable', 0.7916541695594788)]

## Vector Database: **ChromaDB**

In [ ]:
import chromadb
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.create_collection(name="my_collection")

In [ ]:
collection.add(
    embeddings=[[1.2, 2.3, 4.5], [6.7, 8.2, 9.2]],
    documents=["Python is a programming language", "langchain is an framework"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [ ]:
collection.get(ids=["id1"])

{'ids': ['id1'],
 'embeddings': None,
 'metadatas': [{'source': 'my_source'}],
 'documents': ['Python is a programming language'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [ ]:
results = collection.query(
    query_embeddings=[[7.1, 9.1, 6.1]],
    n_results=1
)
results

{'ids': [['id2']],
 'distances': [[10.580000877380371]],
 'metadatas': [[{'source': 'my_source'}]],
 'embeddings': None,
 'documents': [['langchain is an framework']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

## Context Based Generation

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [ ]:
text = r"""
Standard Chartered plc is a British multinational bank with operations in consumer, corporate and institutional banking, and treasury services. Despite being headquartered in the United Kingdom, it does not conduct retail banking in the UK, and around 90% of its profits come from Asia, Africa, and the Middle East.
Standard Chartered has a primary listing on the London Stock Exchange and is a constituent of the FTSE 100 Index. It has secondary listings on the Hong Kong Stock Exchange, the National Stock Exchange of India, and OTC Markets Group Pink. Its largest shareholder is the Government of Singapore-owned Temasek Holdings.[4][5][6] The Financial Stability Board considers it a systemically important bank.
José Viñals is the group chairman of Standard Chartered.[7] Bill Winters is the current group chief executive.[8]
The Standard Bank was a British bank founded in the Cape Province of South Africa in 1862 by Scot, John Paterson.[10] Having established a considerable number of branches Standard was prominent in financing the development of the diamond fields of Kimberley from 1867 and later extended its network further north to the new town of Johannesburg when gold was discovered there in 1885. Half the output of the second largest gold field in the world passed through the Standard Bank on its way to London. Standard expanded widely in Africa over the years, but from 1883 to 1962 was formally known as the Standard Bank of South Africa. In 1962 the bank changed its name to Standard Bank Limited, and the South African operations became a separate subsidiary that took the parent bank's previous name, Standard Bank of South Africa Ltd.[9]
"""


In [ ]:
questions = [
    "Where Standard Chartered Bank is listed? ",
    "Where standard chartered bank is spread?",
    "Who is the current chief of Standard Chartered Bank?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
    input_ids = inputs["input_ids"].numpy()[0]

    outputs = model(inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
    # Get the most likely end of answer with the argmax of the score
    answer_end = tf.argmax(answer_end_scores, axis=1).numpy()[0] + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: Where Standard Chartered Bank is listed? 
Answer: london stock exchange
Question: Where standard chartered bank is spread?
Answer: asia, africa, and the middle east
Question: Who is the current chief of Standard Chartered Bank?
Answer: bill winters
